In [1]:
import numpy as np
import pandas as pd

import itertools
import random

from sklearn.utils import check_random_state

In [2]:
RNG_SEED = 42
rng = check_random_state(RNG_SEED)
print(rng)

RandomState(MT19937)


In [3]:
results1 = 10
results2 = 9
total1 = 0 # np.sum(np.abs(results1))
total2 = 1 # np.sum(np.abs(results2))
percent1 = results1 / total1 if total1 != 0 else 0
percent2 = results2 / total2 if total2 != 0 else 0
percent1, percent2
# return euclidean(results1 / total1, results2 / total2)

(0, 9.0)

In [4]:
index1 = range(0,190)

In [5]:
pairs = list(itertools.combinations(index1, 2))

In [6]:
pair_samples = rng.choice(
        len(pairs),
        size=100,
        replace=False,
    )

pair_samples

array([17326,  9354, 15107,  1554, 11078,  6162,  7858,  6734,  1767,
        8064,  2919,  2885,  8331, 13247,  9690, 14763,  6725,  3877,
        3143,  3971,  4333, 14695, 11282,   971, 17211, 11725, 16467,
       13293, 17930, 14092,   635, 17808,  5026, 15467,  5265,  9724,
       17948,  8769, 10942,  6826, 10958, 17291,  5537, 10689,  3578,
        3369, 17565,  5028, 10730, 16610, 10126,  5849, 16481, 13974,
        6634,  5381,  2495,  3942, 13474, 16182, 10275,  6311,  1355,
        7615,  7576,  5817,  7255,  9509, 11334,  1493,  2325, 11440,
       11283,   290,  7337,  2189, 11980, 17067,  1087,  4634, 10057,
       14757, 15969,  5098,  6286,  2334, 13497, 17114, 14393,  4807,
        2351,  8797, 11257, 17283, 15773,  1020, 11535, 17515, 17623,
        3073])

In [7]:
pairs_sample = [pairs[i] for i in pair_samples]

In [8]:
# [(pair[0], pair[1]) if np.random.choice([0,1]) else (pair[1], pair[0]) for pair in pairs_sample]

In [10]:
sam_idx1 = [i[0] for i in pairs]
sam_idx2 = [i[1] for i in pairs]

In [3]:
import os
import sys

sys.path.append("..")

import tqdm
import time
from copy import deepcopy
import numpy as np
import pandas as pd

import itertools
import random

from sklearn.utils import check_random_state
from lightgbm import LGBMRanker
from sharp import ShaRP
from sharp.utils import scores_to_ordering
from xai_ranking.preprocessing import preprocess_higher_education_data
from xai_ranking.scorers import higher_education_score
from mlresearch.utils import check_random_states

from xai_ranking.preprocessing import (
    preprocess_atp_data,
    preprocess_csrank_data,
    preprocess_higher_education_data,
    preprocess_movers_data,
    preprocess_synthetic_data,
)
from xai_ranking.datasets import (
    fetch_atp_data,
    fetch_csrank_data,
    fetch_higher_education_data,
    fetch_movers_data,
    fetch_synthetic_data,
)
from xai_ranking.scorers import (
    atp_score,
    csrank_score,
    higher_education_score,
    synthetic_equal_score_3ftrs,
)
from xai_ranking.metrics import (
    explanation_sensitivity,
    outcome_sensitivity,
    bootstrapped_explanation_consistency,
    cross_method_explanation_consistency,
    cross_method_outcome_consistency,
    outcome_fidelity,
)

RNG_SEED = 42
N_RUNS = 1

# Set up ranker for the moving company dataset:
X, ranks, score = preprocess_movers_data(fetch_movers_data(test=False))
qids_train = X.index.value_counts().to_numpy()

model = LGBMRanker(
    objective="lambdarank", label_gain=list(range(max(ranks) + 1)), verbose=-1
)
model.fit(
    X=X,
    y=ranks,
    group=qids_train,
)

random_states = check_random_states(RNG_SEED, N_RUNS)

datasets = [
    {
        "name": "Higher Education",
        "data": preprocess_higher_education_data(
            fetch_higher_education_data(year=2020)
        ),
        "scorer": higher_education_score,
        "n_observations": 100,
    },
    {
        "name": "ATP",
        "data": preprocess_atp_data(fetch_atp_data()),
        "scorer": atp_score,
        "n_observations": 86,
    },
    {
        "name": "CSRank",
        "data": preprocess_csrank_data(fetch_csrank_data()),
        "scorer": csrank_score,
        "n_observations": 100,
    },
    # {
    #     "name": "Moving Company",
    #     "data": preprocess_movers_data(fetch_movers_data(test=True)),
    #     "scorer": model.predict,
    #     "n_observations": 100,
    # },
    {
        "name": "Synthetic_0",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=0, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
        "n_observations": 100,
    },
    {
        "name": "Synthetic_1",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=1, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
        "n_observations": 100,
    },
    {
        "name": "Synthetic_2",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=2, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
        "n_observations": 100,
    },
]

# approaches = ["rank", "rank_score", "pairwise-rank", "pairwise-score"]
approaches = ["pairwise-rank", "pairwise-rank_score"]


default_kwargs = {
    "measure": "shapley",
    "sample_size": None,
    "coalition_size": None,
    "replace": False,
    "n_jobs": 14,
}
# parameters_to_change = {
#     "coalition_size": [i for i in range(1, 7)],
#     "sample_size": [20, 50, 100, 250] + list(range(500, 2000, 500)),
#     "n_jobs": [1, 2, 4, 8, 16, 32, 48],
# }

parameters_to_change = {
    "coalition_size": [i for i in range(1, 7)],
    "sample_size": [20, 50, 100, 250] + list(range(500, 2000, 500)),
}

result_cols = (
    [
        "dataset",
        "n_observations",
        "approach",
        "parameter",
        "parameter_value",
        "avg_time",
    ]
    + [f"time_{i}" for i in range(N_RUNS)]
    + [f"agreement_kendall_{i}" for i in range(N_RUNS)]
    + [f"agreement_jaccard2_{i}" for i in range(N_RUNS)]
    + [f"agreement_euclidean_{i}" for i in range(N_RUNS)]
    + [f"fidelity_{i}" for i in range(N_RUNS)]
)


for dataset in datasets:
    result_df = []
    # Set up basic settings
    X = dataset["data"][0]

    # Get scores and ranks
    scorer = dataset["scorer"]
    scores = np.array(scorer(dataset["data"][0]))

    # Remove items that tie
    res = [idx for idx, val in enumerate(scores) if val in scores[:idx]]
    X = X.drop([X.index[i] for i in res])
    # Rescore, get rank
    scores = np.array(scorer(X))
    ranking = scores_to_ordering(scores)

    # Set experiment size if we deleted too many items
    dataset["n_observations"] = dataset["n_observations"] if X.shape[0] > dataset["n_observations"] else X.shape[0]
    
    rng = check_random_state(RNG_SEED)

    # rank and score indexes
    sam_idx = rng.choice(
        np.indices((X.shape[0],)).squeeze(),
        size=dataset["n_observations"],
        replace=False,
    )
    
    # pairwise pairs
    combos = list(itertools.combinations(np.indices((X.shape[0],)).squeeze(), 2))
    pairs_indexes = rng.choice(
        len(combos),
        size=dataset["n_observations"],
        replace=False,
    )
    pairs_sample = [combos[i] for i in pairs_indexes]
    pairs = [(pair[0], pair[1]) if np.random.choice([0,1]) else (pair[1], pair[0]) for pair in pairs_sample]

    for approach in approaches:
        iteration_qoi = approach
        if approach.startswith("pairwise"):
            iteration_qoi = approach.split("-")[1]
            approach = "pairwise"
        print("----------------", dataset["name"], "|", approach, "|", iteration_qoi, "----------------")

        times = []
        kendall_cons = []
        jaccard_cons = []
        euclidean_cons = []
        fidelity = []

        print("Exact computation")
        for i in tqdm.tqdm(range(N_RUNS)):
            start = time.time()
            if approach != "pairwise":
                baseline_sharp = ShaRP(
                    qoi=iteration_qoi,
                    target_function=dataset["scorer"],
                    random_state=random_states[i],
                    **default_kwargs,
                )
                baseline_sharp.fit(X)
                sam_idx1 = sam_idx
                baseline_contr = baseline_sharp.all(X.values[sam_idx1])
            else:
                baseline_sharp = ShaRP(
                    qoi=iteration_qoi,
                    target_function=dataset["scorer"],
                    random_state=random_states[i],
                    **default_kwargs,
                )
                baseline_sharp.fit(X)
                baseline_pairwise = []
                sam_idx1 = [i[0] for i in pairs]
                sam_idx2 = [i[1] for i in pairs]
                for idx1, idx2 in pairs:
                    baseline_pairwise.append(
                        baseline_sharp.pairwise(X.values[idx1], X.values[idx2])
                    )
                baseline_contr = np.array(baseline_pairwise)

            end = time.time()

            baseline_contr = pd.DataFrame(
                baseline_contr, columns=X.columns, index=X.index.values[sam_idx1]
            )
            # Save metrics
            times.append(end - start)
            kendall_cons.append(np.nan)
            jaccard_cons.append(np.nan)
            euclidean_cons.append(np.nan)

            target = scores if approach == "rank_score" else ranking
            avg_target = target.mean()
            if approach != "pairwise":
                res_ = outcome_fidelity(
                    baseline_contr,
                    target[sam_idx1],
                    avg_target,
                    target_max=X.shape[0] if approach == "rank" else target.max(),
                    rank=approach == "rank",
                )
            else:
                res_ = outcome_fidelity(
                    baseline_contr,
                    target[sam_idx1],
                    avg_target,
                    target_max=X.shape[0] if approach == "rank" else target.max(),
                    target_pairs=target[sam_idx2],
                    rank=True,
                )

            fidelity.append(res_)

        exact_results_row = (
            [
                dataset["name"],
                dataset["n_observations"],
                approach,
                np.nan,
                np.nan,
                np.mean(times),
            ]
            + times
            + kendall_cons
            + jaccard_cons
            + euclidean_cons
            + fidelity
        )
        result_df.append(exact_results_row)
        print("Finished computing exact results")
        ############################################################################################

        for parameter, parameter_values in parameters_to_change.items():
            print(f"Alternating parameter: {parameter}")
            default_value = deepcopy(
                default_kwargs[parameter] if parameter in default_kwargs else None
            )

            if parameter == "coalition_size":
                parameter_values = [
                    val for val in parameter_values if X.shape[-1] > val
                ]
            if parameter == "sample_size":
                parameter_values = [
                    val for val in parameter_values if X.shape[0] >= val
                ] + [X.shape[0]]

            if approach == "pairwise" and parameter == "sample_size":
                continue

            for parameter_value in tqdm.tqdm(parameter_values):

                default_kwargs[parameter] = parameter_value

                times = []
                kendall_cons = []
                jaccard_cons = []
                euclidean_cons = []
                fidelity = []

                print(f"Parameter {parameter}, value {parameter_value}")
                for i in tqdm.tqdm(range(N_RUNS)):
                    start = time.time()
                    if approach != "pairwise":
                        sharp = ShaRP(
                            qoi=iteration_qoi,
                            target_function=dataset["scorer"],
                            random_state=random_states[i],
                            **default_kwargs,
                        )
                        sharp.fit(X)
                        sam_idx1 = sam_idx
                        contr = sharp.all(X.values[sam_idx1])
                    else:
                        sharp = ShaRP(
                            qoi=iteration_qoi,
                            target_function=dataset["scorer"],
                            random_state=random_states[i],
                            **default_kwargs,
                        )
                        sharp.fit(X)
                        pairwise = []
                        sam_idx1 = [i[0] for i in pairs]
                        sam_idx2 = [i[1] for i in pairs]
                        for idx1, idx2 in pairs:
                            pairwise.append(
                                sharp.pairwise(X.values[idx1], X.values[idx2])
                            )
                        contr = np.array(pairwise)

                    end = time.time()

                    contr = pd.DataFrame(
                        contr, columns=X.columns, index=np.array(X.index)[sam_idx1]
                    )

                    # Save metrics
                    times.append(end - start)
                    # Kendall consistency
                    kendall_cons.append(
                        cross_method_explanation_consistency(
                            contr, baseline_contr, measure="kendall"
                        )[0]
                    )
                    # Jaccard consistency
                    jaccard_cons.append(
                        cross_method_explanation_consistency(
                            contr, baseline_contr, measure="jaccard", n_features=2
                        )[0]
                    )
                    # Iniatialize normalizer
                    target = scores if approach == "rank_score" else ranking
                    avg_target = target.mean()
                    max_target = X.shape[0] if approach == "rank" else target.max()
                    #Eulidean consistency
                    euclidean_cons.append(
                        cross_method_explanation_consistency(
                            contr, baseline_contr, measure="euclidean", normalizer=max_target
                        )[0]
                    )
                    # Fidelity
                    if approach != "pairwise":
                        res_ = outcome_fidelity(
                            contr,
                            target[sam_idx1],
                            avg_target,
                            target_max=max_target,
                            rank=approach == "rank",
                        )
                    else:
                        res_ = outcome_fidelity(
                            contr,
                            target[sam_idx1],
                            avg_target,
                            target_max=max_target,
                            target_pairs=target[sam_idx2],
                            rank=True,
                        )
        
                    fidelity.append(res_)

                results_row = (
                    [
                        dataset["name"],
                        dataset["n_observations"],
                        approach+"_"+iteration_qoi,
                        parameter,
                        parameter_value,
                        np.mean(times),
                    ]
                    + times
                    + kendall_cons
                    + jaccard_cons
                    + euclidean_cons
                    + fidelity
                )
                result_df.append(results_row)
                print(f"Stored results for {parameter} | {parameter_value}")

            default_kwargs[parameter] = default_value

    results = pd.DataFrame(result_df, columns=result_cols)
    print(results)
    # results.to_csv("notebooks/results/time-experiment-" + dataset["name"] + ".csv")

results = pd.DataFrame(result_df, columns=result_cols)
results

metric = "exp_cons_kendall"
col_mask = results.columns.str.startswith(metric)
results[f"avg_{metric}"] = results.iloc[:, col_mask].mean(1)
col_mask = results.columns == f"avg_{metric}"
col_mask[:6] = True
results.iloc[:, col_mask]

---------------- Higher Education | pairwise | rank ----------------
Exact computation


100%|██████████| 1/1 [00:03<00:00,  3.65s/it]


Finished computing exact results
Alternating parameter: coalition_size


  0%|          | 0/4 [00:00<?, ?it/s]

Parameter coalition_size, value 1


...

Finished computing exact results
Alternating parameter: coalition_size


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter coalition_size, value 1



  0%|          | 0/2 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [5]:
a = [[1,4,3,2], [1,1,1,1], [1,1,2,2], [1,2,2,2], [1,1,2,2], [1,2,3,1], [1,3,2,1], [1,1,2,3],
    [1,2,3,4], [1,1,2,1]]
b = [[1,4,3,2], [1,1,1,1], [1,1,2,2], [1,2,2,2], [2,2,1,1], [1,2,3,1], [1,3,2,1], [1,1,2,3],
    [1,1,3,3], [1,1,1,1]]

In [6]:
import pandas as pd
from sharp.utils import scores_to_ordering
from scipy import stats

def kendall_agreement(results1, results2):
    return results1.reset_index(drop=True).apply(
        lambda row: row_wise_kendall(row, results2.iloc[row.name]), axis=1
    )

def row_wise_kendall(results1, results2):
    results = [results1, results2]

    print("Item1: ", results1.values)
    print("Item2: ", results2.values)
    # Check for ties
    ranks = []
    for result in results:
        rank = scores_to_ordering(result, direction=1)
        # Correct rank values to reflect ties
        for val in result:
            mask = result == val
            if mask.sum() > 1:
                rank[mask] = rank[mask].max()
        ranks.append(rank)

    row_sensitivity = kendall_similarity(*ranks)
    # row_sensitivity = kendall_similarity(*results)
    return row_sensitivity

# def kendall_similarity(a, b):
#     normalizer = (len(a) * (len(a) - 1)) / 2
#     idx_pair = list(combinations(range(len(a)), 2))
#     val_pair_a = [(a[i], a[j]) for i, j in idx_pair if a[i] != a[j]]
#     val_pair_b = [(b[i], b[j]) for i, j in idx_pair if b[i] != b[j]]
#     inversions = sum([(val2, val1) in val_pair_b for val1, val2 in val_pair_a])
#     return (normalizer - inversions) / normalizer

def kendall_similarity(a, b):
    # return 1 - (stats.kendalltau(a, b).statistic + 1) / 2
    print("Rank1: ",a)
    print("Rank2: ",b)
    
    stat = stats.kendalltau(a, b).statistic
    print("Kt-o:", stat)
    print("Kt-n:", 1 - (stat + 1) / 2)
    print("Kt-l", (stat * (-1) + 1) / 2)
    # return stat
    return 1 - (stat + 1) / 2


In [7]:
# a = [[-10,-6,-4,4,6,10],[-10,-10,-10,0,0,0],[-10,-10,-10,0,0,0],[-10,-8,-6,0,6,8], [-10,-10,-10,0,0,0],[-10,-10,-10,0,0,0],
#      [-10,-10,-10,0,0,0]]
# b = [[-10,-6,-4,4,6,10],[-10,-10,-10,0,0,0],[-10,-10,-10,0,0,-10],[8,6,0,-6,-8,-10], [-10,-9,-8,0,1,2],[-10,-10,-9,0,1,0],
#      [-10,-10,-10,0,0,1]]

kendall_agreement(pd.DataFrame(a), pd.DataFrame(b))

Item1:  [1 4 3 2]
Item2:  [1 4 3 2]
Rank1:  [1 4 3 2]
Rank2:  [1 4 3 2]
Kt-o: 1.0
Kt-n: 0.0
Kt-l 0.0
Item1:  [1 1 1 1]
Item2:  [1 1 1 1]
Rank1:  [4 4 4 4]
Rank2:  [4 4 4 4]
Kt-o: nan
Kt-n: nan
Kt-l nan
Item1:  [1 1 2 2]
Item2:  [1 1 2 2]
Rank1:  [2 2 4 4]
Rank2:  [2 2 4 4]
Kt-o: 1.0
Kt-n: 0.0
Kt-l 0.0
Item1:  [1 2 2 2]
Item2:  [1 2 2 2]
Rank1:  [1 4 4 4]
Rank2:  [1 4 4 4]
Kt-o: 1.0
Kt-n: 0.0
Kt-l 0.0
Item1:  [1 1 2 2]
Item2:  [2 2 1 1]
Rank1:  [2 2 4 4]
Rank2:  [4 4 2 2]
Kt-o: -1.0
Kt-n: 1.0
Kt-l 1.0
Item1:  [1 2 3 1]
Item2:  [1 2 3 1]
Rank1:  [2 3 4 2]
Rank2:  [2 3 4 2]
Kt-o: 1.0
Kt-n: 0.0
Kt-l 0.0
Item1:  [1 3 2 1]
Item2:  [1 3 2 1]
Rank1:  [2 4 3 2]
Rank2:  [2 4 3 2]
Kt-o: 1.0
Kt-n: 0.0
Kt-l 0.0
Item1:  [1 1 2 3]
Item2:  [1 1 2 3]
Rank1:  [2 2 3 4]
Rank2:  [2 2 3 4]
Kt-o: 1.0
Kt-n: 0.0
Kt-l 0.0
Item1:  [1 2 3 4]
Item2:  [1 1 3 3]
Rank1:  [1 2 3 4]
Rank2:  [2 2 4 4]
Kt-o: 0.8164965809277261
Kt-n: 0.09175170953613687
Kt-l 0.09175170953613693
Item1:  [1 1 2 1]
Item2:  [1 1 1 1]
Rank1: 

0    0.000000
1         NaN
2    0.000000
3    0.000000
4    1.000000
5    0.000000
6    0.000000
7    0.000000
8    0.091752
9         NaN
dtype: float64

In [8]:
# import pandas as pd
# from sharp.utils import scores_to_ordering
# from itertools import product, combinations

# def kendall_agreement(results1, results2):
#     return results1.reset_index(drop=True).apply(
#         lambda row: row_wise_kendall(row, results2.iloc[row.name]), axis=1
#     )

# def row_wise_kendall(results1, results2):
#     results = [results1, results2]

#     print("Item1: ", results1.values)
#     print("Item2: ", results2.values)
#     # Check for ties
#     ranks = []
#     for result in results:
#         rank = scores_to_ordering(result, direction=1)
#         # Correct rank values to reflect ties
#         for val in result:
#             mask = result == val
#             if mask.sum() > 1:
#                 rank[mask] = rank[mask].max()
#         ranks.append(rank)

#     row_sensitivity = kendall_similarity(*ranks)
#     # row_sensitivity = kendall_similarity(*results)
#     return row_sensitivity

# def kendall_similarity(a, b):
#     print("Rank1: ",a)
#     print("Rank2: ",b)
#     normalizer = (len(a) * (len(a) - 1)) / 2
#     idx_pair = list(combinations(range(len(a)), 2))
#     val_pair_a = [(a[i], a[j]) for i, j in idx_pair if a[i] != a[j]]
#     val_pair_b = [(b[i], b[j]) for i, j in idx_pair if b[i] != b[j]]
#     print(val_pair_a)
#     print(val_pair_b)
#     print([(val2, val1) in val_pair_b for val1, val2 in val_pair_a])
#     inversions = sum([(val2, val1) in val_pair_b for val1, val2 in val_pair_a])
#     kt = 1 - (2 * inversions) / normalizer
#     print("Kt-o:", kt)
#     print("Kt-n:", (normalizer - inversions) / normalizer)
#     return (normalizer - inversions) / normalizer


In [10]:
# a = [[-10,-6,-4,4,6,10],[-10,-10,-10,0,0,0],[-10,-10,-10,0,0,0],[-10,-8,-6,0,6,8], [-10,-10,-10,0,0,0],[-10,-10,-10,0,0,0],
#      [-10,-10,-10,0,0,0],[1,2,3,4,5,6]]
# b = [[-10,-6,-4,4,6,10],[-10,-10,-10,0,0,0],[-10,-10,-10,0,0,-10],[8,6,0,-6,-8,-10], [-10,-9,-8,0,1,2],[-10,-10,-9,0,1,0],
#      [-10,-10,-10,0,0,1],[21,30,12,1,5,3]]

# kendall_agreement(pd.DataFrame(a), pd.DataFrame(b))

In [13]:
import pandas as pd
from sharp.utils import scores_to_ordering
from itertools import product, combinations

def kendall_agreement(results1, results2):
    return results1.reset_index(drop=True).apply(
        lambda row: row_wise_kendall(row, results2.iloc[row.name]), axis=1
    )

def row_wise_kendall(results1, results2):
    results = [results1, results2]

    print("Item1: ", results1.values)
    print("Item2: ", results2.values)
    # Check for ties
    ranks = []
    for result in results:
        rank = scores_to_ordering(result, direction=1)
        # Correct rank values to reflect ties
        for val in result:
            mask = result == val
            if mask.sum() > 1:
                rank[mask] = rank[mask].max()
        ranks.append(rank)

    row_sensitivity = kendall_similarity(*ranks)
    # row_sensitivity = kendall_similarity(*results)
    return row_sensitivity

def kendall_similarity(a, b):
    print("Rank1: ",a)
    print("Rank2: ",b)
    normalizer = (len(a) * (len(a) - 1)) / 2
    idx_pair = list(combinations(range(len(a)), 2))
    val_pair_a = [(a[i], a[j]) for i, j in idx_pair if a[i] != a[j]]
    val_pair_b = [(b[i], b[j]) for i, j in idx_pair if b[i] != b[j]]
    print(val_pair_a)
    print(val_pair_b)
    inversions=0
    for (val11, val12), (val21, val22) in zip(val_pair_a, val_pair_b):
        if ((val11 > val12) and (val21 < val22)) or ((val11 < val12) and (val21 > val22)):
            inversions = inversions+1
    kt = 1 - (2 * inversions) / normalizer
    print("Kt-o:", kt)
    print("Kt-n:", 1 - (kt + 1) / 2)
    return 1 - (kt + 1) / 2

In [14]:
kendall_agreement(pd.DataFrame(a), pd.DataFrame(b))

Item1:  [1 4 3 2]
Item2:  [1 4 3 2]
Rank1:  [1 4 3 2]
Rank2:  [1 4 3 2]
[(1, 4), (1, 3), (1, 2), (4, 3), (4, 2), (3, 2)]
[(1, 4), (1, 3), (1, 2), (4, 3), (4, 2), (3, 2)]
Kt-o: 1.0
Kt-n: 0.0
Item1:  [1 1 1 1]
Item2:  [1 1 1 1]
Rank1:  [4 4 4 4]
Rank2:  [4 4 4 4]
[]
[]
Kt-o: 1.0
Kt-n: 0.0
Item1:  [1 1 2 2]
Item2:  [1 1 2 2]
Rank1:  [2 2 4 4]
Rank2:  [2 2 4 4]
[(2, 4), (2, 4), (2, 4), (2, 4)]
[(2, 4), (2, 4), (2, 4), (2, 4)]
Kt-o: 1.0
Kt-n: 0.0
Item1:  [1 2 2 2]
Item2:  [1 2 2 2]
Rank1:  [1 4 4 4]
Rank2:  [1 4 4 4]
[(1, 4), (1, 4), (1, 4)]
[(1, 4), (1, 4), (1, 4)]
Kt-o: 1.0
Kt-n: 0.0
Item1:  [1 1 2 2]
Item2:  [2 2 1 1]
Rank1:  [2 2 4 4]
Rank2:  [4 4 2 2]
[(2, 4), (2, 4), (2, 4), (2, 4)]
[(4, 2), (4, 2), (4, 2), (4, 2)]
Kt-o: -0.33333333333333326
Kt-n: 0.6666666666666666
Item1:  [1 2 3 1]
Item2:  [1 2 3 1]
Rank1:  [2 3 4 2]
Rank2:  [2 3 4 2]
[(2, 3), (2, 4), (3, 4), (3, 2), (4, 2)]
[(2, 3), (2, 4), (3, 4), (3, 2), (4, 2)]
Kt-o: 1.0
Kt-n: 0.0
Item1:  [1 3 2 1]
Item2:  [1 3 2 1]
Rank1:  [2 4

0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.666667
5    0.000000
6    0.000000
7    0.000000
8    0.000000
9    0.000000
dtype: float64

# Jaccard

In [27]:
from itertools import product, combinations
from sharp.utils import scores_to_ordering

a = [[3.2, -5.1, 4, 0],
    [3.2, -5.1, 4, 0],
    [3.2, -5.1, 4, 0],
    [3.2, -5.1, 4, 0],
    [3.2, 3.2, 3.2, 3.2]]

b = [[3.9, -3.1, 4.1, 0],
    [3.2, -5.1, 3.2, 0],
    [4, -5.1, 3.2, 0],
    [4, -5.1, 3.2, 3.2],
    [3, 3, 3, 3]]


def _get_importance_mask(row_cont, threshold):
    if threshold >= 1:
        # Calculate order of absolute contributions
        row_abs = np.abs(row_cont)
        print(row_abs)
        # Find n=threshold largest items
        res = sorted(row_abs.index.values, key = lambda sub: row_abs[sub])[-threshold:]
        # Set mask
        mask = pd.Series(data=[True if i in res else False for i in row_cont.index.values],
                         index=row_cont.index.values)
    else:
        # Calculate cumulative absolute contribution order
        total_contribution = np.sum(np.abs(row_cont))
        order = np.argsort(np.abs(row_cont))
        cumulative_cont = np.cumsum(np.abs(row_cont)[order]) / total_contribution
        # Find elements withe the smallest contribution
        # (to meet the threshold it's easier to do the reverse operation)
        mask = (cumulative_cont < 1 - threshold)[order]
        # Reverse array
        mask = ~mask

    print(mask)

    # Check whether ties exist
    possible_configs = [mask.copy()]
    tie_values = [
        (idx_old, cont)
        for idx_old, cont in row_cont[mask].items()
        if cont in row_cont[~mask].values
    ]
    print(tie_values)
    # Make all possible sets of ties
    for idx_old, tie_val in tie_values:
        # idx_new = np.where(row_cont == tie_val)[0]
        idx_new = row_cont[row_cont == tie_val].index.values
        print("Substituting:", idx_new )
        # Exclude all selected indexes that have the same value
        idx_new = list(set(idx_new).difference(mask[mask].index.values))
        print(list(set(idx_new)))
        print("Substituting 2:", idx_new )
        for idx in idx_new:
            new_mask = mask.copy()
            new_mask[idx_old] = False
            new_mask[idx] = True
            possible_configs.append(new_mask)

    print("Possible configs:")
    print(possible_configs)
    return possible_configs

def jaccard_similarity(a, b):
    intersection = len(list(set(a).intersection(b)))
    union = (len(set(a)) + len(set(b))) - intersection
    return float(intersection) / union

def row_wise_jaccard(results1, results2, n_features):
    """
    Calculate the row-wise Jaccard similarity between two sets of results.

    Parameters
    ----------
    results1 : numpy.ndarray
        The first set of results. It should be a 2-dimensional array with shape
        (n_samples, n_features).
    results2 : numpy.ndarray
        The second set of results. It should be a 2-dimensional array with shape
        (n_samples, n_features).
    n_features : int, float or None, default=0.8
        The number of top features to consider. If None, all features are
        considered. If an integer value is provided, only the top n_features
        features are considered. If n_features < 1, the most
        important features are determined based on their contribution to the
        total score (as a percentage of the total contribution in absolute
        values).

    Returns
    -------
    float
        The row-wise Jaccard similarity between the two sets of results.

    Notes
    -----
    The row-wise Jaccard similarity is calculated by first converting the
    results into rankings using the `scores_to_ordering` function. Then, the top
    n_features features are selected based on the rankings. Finally, the Jaccard
    similarity is calculated between the selected features for each row.

    If n_features is less than 1, the most important features are determined
    based on their contribution to the total score.  The cumulative contribution
    of each feature is calculated and the features are selected until the
    cumulative contribution exceeds 1 - n_features.

    Examples
    --------
    >>> results1 = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
    >>> results2 = np.array([[0.2, 0.3, 0.4], [0.5, 0.6, 0.7]])
    >>> n_features = 2
    >>> row_wise_jaccard(results1, results2, n_features)
    """

    print("##############")
    print(results1)
    print(results2)
    
    if n_features is None:
        n_features = results1.shape[1]

    masks1 = _get_importance_mask(results1, n_features)
    masks2 = _get_importance_mask(results2, n_features)

    row_similarities = []
    for mask1, mask2 in product(masks1, masks2):
        print("Checking")
        top_idx1 = mask1[mask1].index.values
        top_idx2 = mask2[mask2].index.values
        print(top_idx1)
        print(top_idx2)
        row_similarity = jaccard_similarity(top_idx1, top_idx2)
        row_similarities.append(row_similarity)
    return min(row_similarities)

def jaccard_agreement(results1, results2, n_features=0.8):
    """
    Calculate the Jaccard similarity between two sets of results. Results are
    normalized, 0 means most dis-similar and 1 means most similar.

    Parameters
    ----------
    results1 : pandas.DataFrame
        The first set of results.
    results2 : pandas.DataFrame
        The second set of results.
    n_features : int, float or None, default=0.8
        The number of top features to consider. If None, all features are
        considered. If an integer value is provided, only the top n_features
        features are considered. If n_features < 1, the most
        important features are determined based on their contribution to the
        total score (as a percentage of the total contribution in absolute
        values).

    Returns
    -------
    pandas.Series
        The Jaccard agreement between each pair of results.

    Notes
    -----
    The Jaccard agreement is a measure of similarity between two sets of
    results. It is calculated as the average Jaccard similarity coefficient
    between each pair of results.
    """
    if n_features is None:
        n_features = results1.shape[1]

    return results1.reset_index(drop=True).apply(
        lambda row: row_wise_jaccard(row, results2.iloc[row.name], n_features),
        axis=1,
    )

In [28]:
a = [[-0.428721,-2.801417,6.815626,-0.092408,-1.213119], [-0.428721,-2.801417,6.815626,-0.092408,-2.801417],
     [-2.801417,-2.801417,6.815626,-0.092408,-2.801417]]

b = [[-0.714536,-4.669028,11.359376,-0.154013,-2.021865], [-0.428721,-2.801417,6.815626,-0.092408,-2.801417],
     [-2.801417,-2.801417,6.815626,-0.092408,-2.801417]]

columns=['teaching', 'research', 'citations', 'income', 'international']

jaccard_agreement(pd.DataFrame(a, columns=columns), pd.DataFrame(b, columns=columns), n_features=0.7)

##############
teaching        -0.428721
research        -2.801417
citations        6.815626
income          -0.092408
international   -1.213119
Name: 0, dtype: float64
teaching         -0.714536
research         -4.669028
citations        11.359376
income           -0.154013
international    -2.021865
Name: 0, dtype: float64
research          True
income           False
citations         True
teaching         False
international    False
Name: 0, dtype: bool
[]
Possible configs:
[research          True
income           False
citations         True
teaching         False
international    False
Name: 0, dtype: bool]
research          True
income           False
citations         True
teaching         False
international    False
Name: 0, dtype: bool
[]
Possible configs:
[research          True
income           False
citations         True
teaching         False
international    False
Name: 0, dtype: bool]
Checking
['research' 'citations']
['research' 'citations']
##############
teaching

/state/partition1/job-51014997/ipykernel_2058621/3708206883.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cumulative_cont = np.cumsum(np.abs(row_cont)[order]) / total_contribution
/state/partition1/job-51014997/ipykernel_2058621/3708206883.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mask = (cumulative_cont < 1 - threshold)[order]
/state/partition1/job-51014997/ipykernel_2058621/3708206883.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positio

0    1.000000
1    0.333333
2    0.500000
dtype: float64